<a href="https://colab.research.google.com/github/pietrolira/PPGEEC2318-Redes-Neurais-e-Deep-Learning/blob/main/Lista02/Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**3) Considere quatro distribuições gaussianas, $C_1, C_2, C_3, C_4$, em um espaço de entrada de dimensionalidade igual a oito, isto é $\mathbf{x} = (x_1, x_2, \ldots, x_8)^t$.**

**Visualização 2D com Autoencoder**

Neste notebook, vamos:
1. Gerar 4 distribuições gaussianas de 8 dimensões com médias distintas.
2. Treinar uma rede **autoencoder** para reduzir essas 8 dimensões para 2.
3. Visualizar os dados codificados em 2D.

**a) Todas as nuvens de dados formadas têm variâncias unitárias, mas centros ou vetores média são diferentes e dados por**
$\mathbf{m}_1 = (0,0,0,0,0,0,0,0)^t$,
$\mathbf{m}_2 = (4,0,0,0,0,0,0,0)^t$,
$\mathbf{m}_3 = (0,0,4,0,0,0,0,0)^t$,
$\mathbf{m}_4 = (0,0,0,0,0,0,0,4)^t$.

**Resposta:**
Cada uma das distribuições gaussianas tem variância unitária e diferentes médias dadas por:

* $\mathbf{m}_1 = (0, 0, 0, 0, 0, 0, 0, 0)^T$
* $\mathbf{m}_2 = (4, 0, 0, 0, 0, 0, 0, 0)^T$
* $\mathbf{m}_3 = (0, 0, 4, 0, 0, 0, 0, 0)^T$
* $\mathbf{m}_4 = (0, 0, 0, 0, 0, 0, 0, 4)^T$

**b) Utilizar uma rede de autoencoder para visualizar os dados em duas dimensões.**

**Resposta:**
Foi utilizada uma rede autoencoder para reduzir a dimensionalidade dos dados para duas dimensões (2d), ou seja, uma rede neural que aprenda a codificar os vetores de 8 dimensões em um espaço de 2 dimensões e consiga reconstruir os dados originais a partir disso.

**c) O objetivo é visualizar os dados de dimensão 8 em um espaço de dimensão 2.**

**Resposta:**
O objetivo principal é visualizar os dados originalmente em 8 dimensões em um novo espaço bidimensional (2D), preservando suas estruturas e relações.


**d) Apresente os dados neste novo espaço.**

**Resposta:**
Os dados foram apresentados no novo espaço bidimensional, mostrando as quatro classes de forma visualmente separável, onde cada ponto representa um vetor codificado e cores distintas indicam cada uma das classes $C_1, C_2, C_3, C_4$.

---

**Código Python**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# **Fixar semente para reprodutibilidade**
np.random.seed(42)
torch.manual_seed(42)

**Criar as quatro distribuições gaussianas**


In [ ]:
means = [
    np.array([0, 0, 0, 0, 0, 0, 0, 0]),
    np.array([4, 0, 0, 0, 0, 0, 0, 0]),
    np.array([0, 0, 4, 0, 0, 0, 0, 0]),
    np.array([0, 0, 0, 0, 0, 0, 0, 4])
]

num_samples_per_class = 500
X, y = [], []

# Gerar amostras com variância unitária
for i, mean in enumerate(means):
    cov = np.eye(8)
    samples = np.random.multivariate_normal(mean, cov, num_samples_per_class)
    X.append(samples)
    y.append(np.full(num_samples_per_class, i))

X = np.vstack(X)
y = np.concatenate(y)

# Converter para tensores

X_tensor = torch.tensor(X, dtype=torch.float32)

**Definir Autoencoder**

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 8)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon

# Instanciar modelo
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Treinamento
epochs = 200
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, X_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

**Obter representações em 2D**


In [ ]:
with torch.no_grad():
    encoded = model.encoder(X_tensor).numpy()

**Visualizar**


In [ ]:
plt.figure(figsize=(8, 6))
colors = ['red', 'green', 'blue', 'orange']
labels = ['C1', 'C2', 'C3', 'C4']

for i in range(4):
    plt.scatter(encoded[y == i, 0], encoded[y == i, 1],
                label=labels[i], alpha=0.6, color=colors[i])

plt.title('Visualização 2D com Autoencoder')
plt.xlabel('Dimensão 1')
plt.ylabel('Dimensão 2')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()